# Simple char-RNN 실습

![img](../assets/simple_char_rnn.png)


In [1]:
import numpy as np
import torch
import torch.nn as nn
import sys

## One hot encoding
훈련시키고자 하는 character를 수치화하는 방법

In [2]:
h = [1,0,0,0]
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]

## RNN cell 생성
- input_dim = 4 <-- one hot encoding  
- hidden_size = 2 <-- output으로 나오는 갯수

In [3]:
cell = nn.RNN(input_size = 4, hidden_size=2, batch_first=True)

In [4]:
inputs = torch.tensor([[h]], dtype=torch.float32) # <- 문자 1개 
print(inputs.size())

torch.Size([1, 1, 4])


In [5]:
hidden = torch.randn(1,1,2) # init hidden은 랜덤값으로 

In [6]:
out, hidden = cell(inputs, hidden)
print(out.data)

tensor([[[-0.6755,  0.6619]]])


## Using Sequence
input값을 넣을때 단일 문자만을 넣는게 아니라 여러개의 문자를 입력으로 준다.  
즉, 단일한 문자 여러개를 하나의 sequence로 묶어 입력으로 제공한다.  
기본적으로 위의 예와 같고 입력에서 단일값이 아니라 문자를 여러개 입력으로 준다.
- hidden_size = 2
- **seq_len = 5**

In [7]:
cell = nn.RNN(input_size = 4, hidden_size=2, batch_first=True)

inputs = torch.tensor([[h,e,l,l,o]], dtype=torch.float32) # <- 문자 n개를 sequence로 준다.
print("input_size :",inputs.size())

hidden = torch.randn(1,1,2) # init hidden은 랜덤값으로 

out, hidden = cell(inputs, hidden)
print(out.data)

input_size : torch.Size([1, 5, 4])
tensor([[[-0.4777, -0.3481],
         [-0.7116, -0.3802],
         [-0.6112,  0.5556],
         [-0.0536, -0.0232],
         [-0.2218,  0.0640]]])


## Batch size
앞의 예제에 더 붙여 한 번에 하나의 단어만 입력하는 것이 아니라 여러개의 단어(= character의 sequence)를 입력하기 위해서 batch_size를 준다.

- hidden_size = 2
- seq_len = 5
- **batch_size = 3**

In [8]:
cell = nn.RNN(input_size = 4, hidden_size=2, batch_first=True)

inputs = torch.tensor([[h,e,l,l,o],
                       [e,o,l,l,l],
                       [l,l,e,e,l]], dtype=torch.float32) # <- 문자 n개를 sequence로 준다.
print("input_size :",inputs.size())

hidden = torch.randn(1,3,2) # init hidden은 랜덤값으로 , batch_size를 맞추라! 

out, hidden = cell(inputs, hidden)
print(out.data)

input_size : torch.Size([3, 5, 4])
tensor([[[-0.6966,  0.3133],
         [ 0.0866,  0.6837],
         [-0.7175,  0.7736],
         [-0.6753,  0.7137],
         [-0.7392,  0.1297]],

        [[ 0.1937,  0.5673],
         [-0.7728,  0.2524],
         [-0.6082,  0.6009],
         [-0.6632,  0.6937],
         [-0.6700,  0.7050]],

        [[-0.6686,  0.6897],
         [-0.6691,  0.7037],
         [ 0.0029,  0.7544],
         [-0.0914,  0.8148],
         [-0.7198,  0.7778]]])


## chararater-level RNN


![img](../assets/simple_char_rnn1.png)

hihell을 입력하면 ihello를 생성하는 RNN 모형을 생성하자.  
hihello의 unique 값 --> h, i, e, l, o ... 5개이다.  
이것을 one hot encoding하면  
h --> [1,0,0,0,0]  
i --> [0,1,0,0,0]  
e --> [0,0,1,0,0]  
l --> [0,0,0,1,0]  
o --> [0,0,0,0,1]  

- input_dim = 5 
- output_dim = 5

## loss function
multi class classification 이므로 cross_entropy를 쓴다.  

## 데이터 전처리 1

In [9]:
idx2char = ['h','i','e','l','o'] # 사용할 기본 voca에 해당, unique 문자

x_data = [0,1,0,2,3,3] # 입력 hihell --> 위에 생성한 voca의 index값으로 입력

x_one_hot = [[[1,0,0,0,0], # h
              [0,1,0,0,0], # i
              [1,0,0,0,0], # h
              [0,0,1,0,0], # e
              [0,0,0,1,0], # l
              [0,0,0,1,0]  # l
              ]]

y_data = [1,0,2,3,3,4] # ihello를 출력하게 --> index값으로 출력

x_one_hot = []

inputs = torch.tensor(x_one_hot, dtype=torch.float32) # input은 반드시 one-hot vector가 되어야
labels = torch.tensor(y_data)

print(inputs.type(), labels.type())


torch.FloatTensor torch.LongTensor


## 데이터 전처리 2
앞의 예와 동일한데 다만 다른 점은 one_hot_lookup을 만들고 이로 부터 x_one_hot을 만드는 부분이 다르다.

In [10]:
idx2char = ['h','i','e','l','o'] # 사용할 기본 voca에 해당, unique 문자

x_data = [0,1,0,2,3,3] # 입력 hihell --> 위에 생성한 vaca의 index값으로 입력


# index 값으로 lookup 을 생성한다. 
one_hot_lookup = [ [1,0,0,0,0], # 0
                   [0,1,0,0,0], # 1
                   [0,0,1,0,0], # 2
                   [0,0,0,1,0], # 3
                   [0,0,0,1,0] # 4
                 ]

y_data = [1,0,2,3,3,4] # ihello를 출력하게 --> index값으로 출력

x_one_hot = [one_hot_lookup[x] for x in x_data] # 이 부분이 다르다. loopkup으로 부터 one-hot vector 생성

inputs = torch.tensor(x_one_hot, dtype=torch.float32) # input은 반드시 one-hot vector가 되어야
labels = torch.tensor(y_data).unsqueeze(1)

#print(inputs.size(), labels.size())


## define the model

In [13]:
class Model(nn.Module) :
    def __init__(self, input_size, hidden_size,) :
        super(Model, self).__init__()
        self.rnn = nn.RNN(input_size=input_size, 
                          hidden_size=hidden_size, batch_first=True)
    
    def forward(self, x, hidden) :
        x = x.view(batch_size, sequence_length, input_size)
        
        # Input: (batch, seq_len, input_size)
        # hidden: (num_layers * num_directions, batch, hidden_size)
        out, hidden = self.rnn(x, hidden)
        
        out = out.view(-1, num_classes) 
        
        return out, hidden

    def init_hidden(self) :
        return torch.zeros(num_layers, batch_size, hidden_size)

## 모형의 생성

In [28]:
num_classes = 5
input_size = 5  # one-hot size
hidden_size = 5  # output from the RNN. 5 to directly predict one-hot
batch_size = 1   # one sequence
sequence_length = 1  # One by one
num_layers = 1  # one-layer rnn

# Instantiate RNN model
model = Model(input_size, hidden_size)

print(model)

Model(
  (rnn): RNN(5, 5, batch_first=True)
)


## Loss & Optimizer

In [29]:
# Set loss and optimizer function
# CrossEntropyLoss = LogSoftmax + NLLLoss
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

## 모형의 훈련

In [30]:
# Train the model

epochs = 50 

for epoch in range(epochs):
    optimizer.zero_grad()
    loss = 0
    hidden = model.init_hidden()

    print("predicted string : ", end='')
    for input, label in zip(inputs, labels):
        output, hidden = model(input, hidden)
        val, idx = output.max(1)
        print(idx2char[idx.item()],end='')
        loss += criterion(output, label)

    loss.backward()
    optimizer.step()

    print(", epoch: %d, loss: %1.3f" % (epoch + 1, loss.item()))


predicted string : llllll, epoch: 1, loss: 9.317
predicted string : llllll, epoch: 2, loss: 9.201
predicted string : llllll, epoch: 3, loss: 9.088
predicted string : lhllll, epoch: 4, loss: 8.977
predicted string : lhllll, epoch: 5, loss: 8.868
predicted string : lhllll, epoch: 6, loss: 8.760
predicted string : lhllll, epoch: 7, loss: 8.653
predicted string : lhllll, epoch: 8, loss: 8.548
predicted string : lhllll, epoch: 9, loss: 8.445
predicted string : lhelll, epoch: 10, loss: 8.343
predicted string : lhelll, epoch: 11, loss: 8.242
predicted string : lhelll, epoch: 12, loss: 8.143
predicted string : lhelll, epoch: 13, loss: 8.045
predicted string : lhelll, epoch: 14, loss: 7.948
predicted string : lhelll, epoch: 15, loss: 7.853
predicted string : lhelll, epoch: 16, loss: 7.759
predicted string : lhelll, epoch: 17, loss: 7.666
predicted string : lhelll, epoch: 18, loss: 7.575
predicted string : ihelll, epoch: 19, loss: 7.483
predicted string : ihelll, epoch: 20, loss: 7.393
predicted